# **DIMEMEX — Complete Project Pipeline Notebook**

*Multilingual Meme Translation & Hate Speech Analysis*

---

## **Members**

* **Bárbara** (Text)
* **Amanda** (Text + Description)
* **Juan David Nieto** (Text + Description + Image)
* **Luisa** (Image)

---

## **Project Overview**

This project analyzes whether **offensive or hate speech content in memes is preserved after translating the memes from Spanish to Portuguese**.

We work with the **DIMEMEX** dataset, which contains:

* Meme **text**
* Meme **description**
* Meme **image**
* Labels: *hate speech*, *inappropriate content*, *neither*

This project has **two main tasks**:

### **Task 1 — Translation Quality Evaluation**

Translate the Spanish text to Portuguese and evaluate translation quality using standard NLP metrics:

* **BLEURT**
* **BERTScore**
* **COMET-Kiwi**
* **chrF**

### **Task 2 — Hate Speech Detection (Multimodal Fine-Tuning)**

Fine-tune models to classify:

* Hate speech
* Inappropriate content
* Neither

We fine-tune under four input settings:

1. Text
2. Text + Description
3. Text + Description + Image (Multimodal)
4. Image

---

## **📌 Objectives**

1. Evaluate whether offensive content is **maintained or lost** during translation.
2. Compare performance of **original Spanish memes vs. translated Portuguese memes**.
3. Train and evaluate multimodal detectors to classify hate speech.
4. Analyze cases where the label changes across languages.
5. Perform **human qualitative analysis** on inconsistent cases.



# Imports

# 🛠️ 1. Install Dependencies

In [4]:
!pip install -q transformers accelerate datasets peft bitsandbytes torch torchvision pillow tqdm scikit-learn matplotlib evaluate
!pip install -q trl # Install TRL for SFTTrainer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 14.2 MB/s eta 0:00:00


In [5]:
import os
import zipfile
import json
import warnings

import pandas as pd
import numpy as np
from PIL import Image

from google.colab import files

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

import torch

from datasets import Dataset
import evaluate
from evaluate import load

# Hugging Face - Transformers (Models, Processors, Configs)
from transformers import (
    AutoTokenizer,
    AutoProcessor,
    AutoModelForCausalLM,
    Idefics3ForConditionalGeneration,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)

# Hugging Face - PEFT (LoRA)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel
)

# Hugging Face - TRL (Training)
from trl import SFTTrainer

from tqdm import tqdm
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# 🗂️ 2. Upload Data

In [2]:
from google.colab import drive
import os
import pandas as pd

# Montar Drive
drive.mount('/content/drive')

# === Caminhos dos CSVs (ajuste se preferir) ===
CSV_TRAIN = "/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX/train/dados_espanhol.csv"
CSV_VAL   = "/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX/validation/dados_espanhol.csv"
CSV_TEST  = "/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX/test/dados_espanhol.csv"

# === Caminhos das pastas de imagens ===
TRAIN_IMAGES_DIR = "/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX/train_images"
VAL_IMAGES_DIR   = "/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX/validation_images"
TEST_IMAGES_DIR  = "/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX/test_images"

# Carregar CSVs
df_train = pd.read_csv(CSV_TRAIN)
df_val   = pd.read_csv(CSV_VAL)
df_test  = pd.read_csv(CSV_TEST)

print("CSV train:", df_train.shape)
print("CSV val:", df_val.shape)
print("CSV test:", df_test.shape)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CSV train: (2262, 4)
CSV val: (322, 4)
CSV test: (648, 4)


# ⚙️ 4. Main Configurations

In [7]:
# --- Model & Training Params ---
MODEL_ID = "HuggingFaceTB/SmolVLM-256M-Instruct"
BATCH_SIZE = 2
EPOCHS = 2
LR = 2e-4
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- Fixed Label Mapping (Luisa's desired order) ---
label_to_id = {
    "hate_speech": 0,
    "inappropriate": 1,
    "neither": 2,
}

id_to_label = {v: k for k, v in label_to_id.items()}

print("Label mapping:", label_to_id)
print("\nQuantidade de exemplos por label:")
print(df_train["label"].value_counts())


Label mapping: {'hate_speech': 0, 'inappropriate': 1, 'neither': 2}

Quantidade de exemplos por label:
label
hate speech              1404
inappropriate content     472
neither                   386
Name: count, dtype: int64


#📦 5. Prepare Train, Validation, and Test Datasets

In [8]:
# Os dados já estão divididos

df_train["image_path"] = df_train["image_path"].apply(lambda x: os.path.join(TRAIN_IMAGES_DIR, x))
df_val["image_path"]   = df_val["image_path"].apply(lambda x: os.path.join(VAL_IMAGES_DIR, x))
df_test["image_path"]  = df_test["image_path"].apply(lambda x: os.path.join(TEST_IMAGES_DIR, x))

ds_train = Dataset.from_pandas(df_train.reset_index(drop=True))
ds_val = Dataset.from_pandas(df_val.reset_index(drop=True))
ds_test = Dataset.from_pandas(df_test.reset_index(drop=True))

print(f"Train: {len(ds_train)} | Validation: {len(ds_val)} | Test: {len(ds_test)}")

Train: 2262 | Validation: 322 | Test: 648


#🧩 6. Processor and Pre-processing **(Text + Desc + Image)**

In [9]:
processor = AutoProcessor.from_pretrained(MODEL_ID)

def format_for_sft(example):
    """
    Formats the example for SFT training for SmolVLM.
    """

    # --- Load image safely ---
    path = example["image_path"]
    if not os.path.exists(path):
        image = Image.new("RGB", (100, 100), "gray")
    else:
        image = Image.open(path).convert("RGB")

    # --- Construct the multimodal prompt ---
    prompt_text = (
        f"Analyze the following meme components and classify it. "
        f"Text: {example['text']}\n"
        f"Description: {example['description']}\n\n"
        f"What is the category?"
    )

    # --- Chat format for SmolVLM ---
    messages = [
        {"role": "user", "content": [image, prompt_text]},
        {"role": "assistant", "content": str(label_to_id[example['label']])}
    ]

    # Apply template → creates "text" for training
    example["text"] = processor.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    return example

# --- Apply formatting ---
ds_train = ds_train.map(format_for_sft, remove_columns=ds_train.column_names)
ds_val   = ds_val.map(format_for_sft, remove_columns=ds_val.column_names)


processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/429 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/2262 [00:00<?, ? examples/s]

KeyError: 'hate speech'

# 🧠 7. Model + LoRA

In [ ]:
# 4-bit Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = Idefics3ForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

base_model = prepare_model_for_kbit_training(base_model)

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    # Target modules updated for Idefics3/SmolVLM
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, lora_cfg)
model.print_trainable_parameters()

# 🧮 8. Training Configuration

In [ ]:
OUTPUT_DIR = "./SmolVLM_256M_Instruct_DIMEMEX_lora"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=False, # Must be False for 4-bit
    bf16=True,  # Use bf16 for 4-bit
    load_best_model_at_end=True,
    logging_steps=50,
    save_total_limit=3,
    report_to="none", # Disable wandb/tensorboard logging
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    tokenizer=processor, # The processor handles tokenization
    dataset_text_field="text", # The column we created in Section 6
    max_seq_length=1024,
)

# 10. Train Model

In [ ]:
trainer.train()
print("Fine-tuning complete.")

# 11. Loss Curves

In [ ]:
log_file = os.path.join(training_args.output_dir, "checkpoint-XXX", "trainer_state.json")
# Note: You may need to update 'checkpoint-XXX' to the latest checkpoint folder
# For simplicity, we'll access the trainer's state directly.

logs = trainer.state.log_history

train_steps = [x["step"] for x in logs if "loss" in x]
train_loss = [x["loss"] for x in logs if "loss" in x]

eval_steps = [x["step"] for x in logs if "eval_loss" in x]
eval_loss = [x["eval_loss"] for x in logs if "eval_loss" in x]

plt.figure(figsize=(10, 5))
plt.plot(train_steps, train_loss, label="Train Loss")
plt.plot(eval_steps, eval_loss, label="Eval Loss", marker='o')
plt.title("Training and Evaluation Loss")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.legend()
plt.show()

# ✅ 12. Final Evaluation and Metrics Summary

In [ ]:
# Use the fine-tuned model (already loaded in 'trainer.model')
model = trainer.model
test_predictions = []
test_ground_truth = []

# Loop through the test set
print("Running final evaluation on test set...")
for example in tqdm(ds_test):
    # Re-create the prompt, but without the answer
    prompt_text = (
        f"Analyze the following meme components and classify it. "
        f"Text: {example['text']}\n"
        f"Description: {example['description']}\n\n"
        f"What is the category? "
    )
    image = Image.open(example['image_path']).convert("RGB")
    messages = [{"role": "user", "content": [image, prompt_text]}]

    # Prepare input
    prompt = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)

    # Generate response
    output = model.generate(**inputs, max_new_tokens=20, do_sample=False)
    generated_text = processor.decode(output[0], skip_special_tokens=True)

    # Extract just the answer (the label)
    prediction = generated_text.split("assistant\n")[-1].strip()

    test_predictions.append(prediction)
    test_ground_truth.append(example['label'])

# --- Compute Metrics ---
# Filter out any predictions that were not valid labels
valid_preds = [p for p in test_predictions if p in label_to_id]
valid_gt = [g for p, g in zip(test_predictions, test_ground_truth) if p in label_to_id]

print(f"\n{len(valid_preds)} / {len(test_predictions)} predictions were valid labels.")

print("\n📊 Final Test Set Results:")
print(classification_report(valid_gt, valid_preds, target_names=label_to_id.keys()))

# Manual calculation for overall results
results = {}
results["accuracy"] = accuracy_score(valid_gt, valid_preds)
results["f1_weighted"] = f1_score(valid_gt, valid_preds, average="weighted")
results["precision_weighted"] = precision_score(valid_gt, valid_preds, average="weighted", zero_division=0)
results["recall_weighted"] = recall_score(valid_gt, valid_preds, average="weighted", zero_division=0)

print("\n📊 Summary Metrics:")
for k, v in results.items():
    print(f"{k:20s}: {v:.4f}")

# 💾 13. Save Final Model

In [ ]:
# Save the LoRA adapters
model.save_pretrained(OUTPUT_DIR)
# Save the processor
processor.save_pretrained(OUTPUT_DIR)

print(f"\n✅ Fine-tuning complete. Model saved to: {OUTPUT_DIR}")